# Deep Learning Lab using PyTorch and Common Helper Packages

Courtesy of ISLP

# Imports

General Imports

In [2]:
import numpy as np , pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
(LinearRegression ,
LogisticRegression ,
Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
(train_test_split ,
GridSearchCV)

Torch specific imports

In [3]:
# General torch imports
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

# torchmetrics, for further performance metrics
from torchmetrics import (MeanAbsoluteError, R2Score)
from torchinfo import summary
from torchvision.io import read_image

# pytorch lightning, for interface used for training
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

# for reproducibility,
# from pytorch_lightning.utilities.seed import seed_everything
# seed_everything (0, workers=True)
torch.use_deterministic_algorithms(True , warn_only=True)

Load data

In [4]:
from torchvision.datasets import MNIST , CIFAR100
from torchvision.models import (resnet50 ,
    ResNet50_Weights)
from torchvision.transforms import (Resize ,
    Normalize ,
    CenterCrop ,
    ToTensor)

ISLP specific imports

In [5]:
from ISLP.torch import (SimpleDataModule ,
SimpleModule ,
ErrorTracker ,
rec_num_workers)

from ISLP.torch.imdb import (load_lookup ,
load_tensor ,
load_sparse ,
load_sequential)

Misc imports

In [6]:
from glob import glob
import json

# Feed Forward NN's

## Single Layer NN

In [8]:
# load data
Hitters = load_data('Hitters').dropna()
n = Hitters.shape[0]
print(Hitters.shape)

(263, 20)


In [10]:
# use MS from ISLP to create a design matrix, then convert to a numpy array
model = MS(Hitters.columns.drop('Salary'), intercept=False)
X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary'].to_numpy()

# split into training and test
(X_train, X_test, Y_train, Y_test) = train_test_split(
    X, Y, test_size=1/3, random_state=1
)

Fit a linear model as a very quick baseline

In [21]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat = hit_lm.predict(X_test)
np.abs(Yhat - Y_test).mean()

259.71528833146243

Fit a lasso regression as a slightly less quick baseline

In [24]:
# scale data for the regularisation, then apply the lasso regularisation
# the Pipeline just combines these 2 steps in one for us
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000)
standard_lasso = Pipeline(steps=[
    ('scaler', scaler),
    ('lasso', lasso)
])

# set up a grid of lambda values for lasso
X_scaled = scaler.fit_transform(X_train)
n = X_scaled.shape[0]
lam_max = np.fabs(X_scaled.T.dot(Y_train - Y_train.mean()).max()) / n
param_grid = {'alpha': np.exp(np.linspace(0, np.log(0.01), 100)) * lam_max}

In [25]:
cv = KFold(10, shuffle=True, random_state=1)
grid = GridSearchCV(lasso, param_grid=param_grid, cv=cv, scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
             estimator=Lasso(max_iter=30000, warm_start=True),
             param_grid={'alpha': array([255.65755026, 244.03752004, 232.94563812, 222.35789935,
       212.25138966, 202.60423642, 193.39556119, 184.60543446,
       176.21483255, 168.20559645, 160.5603925 , 153.26267486,
       146.29664975, 139.64724123, 133.3000586 , 127.24136521,
       121.45...
        10.81237873,  10.32093943,   9.8518368 ,   9.40405561,
         8.97662677,   8.56862523,   8.17916799,   7.80741218,
         7.45255325,   7.1138232 ,   6.79048896,   6.48185076,
         6.18724064,   5.906021  ,   5.63758323,   5.38134637,
         5.13675587,   4.90328239,   4.68042064,   4.4676883 ,
         4.26462497,   4.07079118,   3.88576744,   3.70915331,
         3.54056657,   3.37964236,   3.2260324 ,   3.07940424,
         2.93944057,   2.80583846,   2.67830877,   2.5565755 ])},
             scoring='neg_mean_absolute_error')

In [26]:
lasso_model = grid.best_estimator_
Yhat = lasso_model.predict(X_test)
np.fabs(Yhat - Y_test).mean()

257.2382010799497

Neural net time

In [12]:
# nn blueprint
class HittersModel(nn.Module):
    def __init__(self, input_size):
        super(HittersModel, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))


hit_model = HittersModel(X.shape[1])

summary(hit_model, input_size=X_train.shape, col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
HittersModel                             [175, 19]                 [175]                     --
├─Flatten: 1-1                           [175, 19]                 [175, 19]                 --
├─Sequential: 1-2                        [175, 19]                 [175, 1]                  --
│    └─Linear: 2-1                       [175, 19]                 [175, 50]                 1,000
│    └─ReLU: 2-2                         [175, 50]                 [175, 50]                 --
│    └─Dropout: 2-3                      [175, 50]                 [175, 50]                 --
│    └─Linear: 2-4                       [175, 50]                 [175, 1]                  51
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
Total mult-adds (M): 0.18
Input size (MB): 0.01
Forward/backward pass size (MB): 0.07
Params size (MB): 0.00
Estimated Total Size (MB): 0.09

In [14]:
# torch requires our data to be in the tensor datatype, and in 32 bit. we convert it to this form
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.float32))
hit_train = TensorDataset(X_train_t, Y_train_t)

X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))
hit_test = TensorDataset(X_test_t, Y_test_t)

In [22]:
# training 
max_num_workers = rec_num_workers()

hit_dm = SimpleDataModule(
    hit_train, hit_test, batch_size=32,
    num_workers = min(4, max_num_workers), validation=hit_test
)

hit_module = SimpleModule.regression(hit_model,
                                     metrics={'mae':MeanAbsoluteError()})

hit_logger = CSVLogger('logs', name='hitters')

hit_trainer = Trainer(
    deterministic=True,
    max_epochs=100,
    log_every_n_steps=5,
    logger=hit_logger,
    callbacks=[ErrorTracker()]
)

hit_trainer.fit(hit_module, datamodule=hit_dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | HittersModel | 1.1 K  | eval 
1 | loss  | MSELoss      | 0      | train
-----------------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)
1         Modules in train mode
7         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [23]:
# testing
hit_trainer.test(hit_module, datamodule=hit_dm)

hit_model.eval()
preds = hit_module(X_test_t)
torch.abs(Y_test_t - preds).mean()

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss               97430.53125
        test_mae            182.86878967285156
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


tensor(182.8688, grad_fn=<MeanBackward0>)

In [24]:
# kill all processes
del(Hitters ,
hit_model , hit_dm ,
hit_logger ,
hit_test , hit_train ,
X, Y,
X_test , X_train ,
Y_test , Y_train ,
X_test_t , Y_test_t ,
hit_trainer , hit_module)

## Multilayer Network on MNIST